# Add predicted domains DUFLT

Version février 2026 intégrant les 40 domaines de Denis

In [1]:
from transformers import BertForSequenceClassification, AutoTokenizer
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import torch
from tqdm.auto import tqdm
import os
import datetime

# date as "yyyymmdd"
suffix = datetime.datetime.now().strftime("%y%m%d")
suffix

'260220'

In [2]:

# 1. Setup Device
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

TEXT_COLUMN = "text"

# 2. Load Model and Tokenizer
MODEL_PATH = "../models/final_multiclass_model"
model = BertForSequenceClassification.from_pretrained(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model.to(device)
model.eval() # Set to evaluation mode

# 3. Create a simple Dataset class
class SimpleDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

def predict_dataframe(df, text_column, batch_size=16):
    texts = df[text_column].astype(str).tolist()
    dataset = SimpleDataset(texts)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    
    all_preds = []
    
    # Wrap dataloader with tqdm for the progress bar
    # 'total' helps tqdm calculate the percentage correctly
    progress_bar = tqdm(dataloader, desc="Predicting", unit="batch")
    
    with torch.no_grad():
        for batch in progress_bar:
            inputs = tokenizer(
                batch, 
                padding=True, 
                truncation=True, 
                max_length=512, 
                return_tensors="pt"
            ).to(device)
            
            outputs = model(**inputs)
            preds = torch.argmax(outputs.logits, axis=-1).cpu().numpy()
            all_preds.extend(preds)
            
            # Optional: Add real-time info to the bar
            # progress_bar.set_postfix({"last_batch_size": len(preds)})

    df['predicted_label_id'] = all_preds
    df['predDomain'] = df['predicted_label_id'].apply(lambda x: model.config.id2label[x])
    return df



In [3]:
sentence = "Brin , se dit aussi des menus jets des herbes, des joncs, des cheveux, & de tout ce que des racines poussent. Coliculus, surculus. Il faut mettre deux ou trois brins de ciboulette dans cette salade. Il n'est resté à ce convalescent que deux ou trois brins de cheveux. Les brins des vergettes sont faits de petits joncs. Les tresses de cheveux se font brin à brin. Philyra."

inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
outputs = model(**inputs)
pred = torch.argmax(outputs.logits, axis=-1).cpu().item()
pred_label = model.config.id2label[pred]
print(f"Predicted label: {pred_label} (ID: {pred})")

Predicted label: Métiers [alimentation] (ID: 26)


In [21]:
#filepath = '/Users/lmoncla/Nextcloud/GEODE/GEODE - Partage consortium/Corpus/EDdA/EDdA_dataset_articles_250517.tsv'
filepath = '/Users/lmoncla/Nextcloud/GEODE/GEODE - Partage consortium/Corpus/EDdA/EDdA_dataset_paragraphs_230407.tsv'

df = pd.read_csv(filepath, sep='\t', encoding='utf-8')
df.rename(columns={"edda_class":"srcDomain", "paragraph_id":"paragrapId", "content":"text"}, inplace=True)
df.drop(columns=['author', "nb_place_spacy", "nb_person_spacy"], inplace=True)
df['text'] = df['text'].str.replace("\n", "")
#df = pd.read_csv(os.path.join('..', 'data', '1743_LeRobert', 'Trevoux1743_html_macro_260213.tsv'), sep='\t', encoding='utf-8')
df.head()

,volume,numero,head,srcDomain,paragrapId,text
0,1,1,Title Page,unclassified,1,"ENCYCLOPÉDIE,DICTIONNAIRE RAISONNÉDES SCIENCES..."
1,1,1,Title Page,unclassified,2,"Mis en ordre & publié par M. DIDEROT, de l'Aca..."
2,1,1,Title Page,unclassified,3,"Tantùm series juncturaque pollet,Tantùm de med..."
3,1,1,Title Page,unclassified,4,"A PARIS,Chez BRIASSON, rue Saint Ja..."
4,1,1,Title Page,unclassified,5,M. DCC. L I.AVEC APPROBATION ET PRIVILEGE DU ROY.


In [22]:
df.shape

(261633, 6)

In [23]:
# get a sample of rows where text is longer than 100 characters or src-domain is not null
# DUFLT
#df_sample = df[(df['text'].str.len() > 100) | (df['srcDomain'].isna() == False)]

# EDdA
df_sample = df[(df['text'].str.len() > 100) | ((df['srcDomain'].isna() == False) & (df['srcDomain'] != 'pending') & (df['srcDomain'] != '0') & (df['srcDomain'] != 'unclassified'))]

df_sample

,volume,numero,head,srcDomain,paragrapId,text
0,1,1,Title Page,unclassified,1,"ENCYCLOPÉDIE,DICTIONNAIRE RAISONNÉDES SCIENCES..."
1,1,1,Title Page,unclassified,2,"Mis en ordre & publié par M. DIDEROT, de l'Aca..."
3,1,1,Title Page,unclassified,4,"A PARIS,Chez BRIASSON, rue Saint Ja..."
7,1,2,A MONSEIGNEUR LE COMTE D'ARGENSON,unclassified,3,L'autorité suffit à un Ministre pour lui attir...
12,1,3,DISCOURS PRÉLIMINAIRE DES EDITEURS,unclassified,2,"L'Encyclopédie que nous présentons au Public, ..."
...,...,...,...,...,...,...
261628,17,3208,"VINGTIEME, imposition",Economie politique,498,Elle seroit déposée dans chaque communauté com...
261629,17,3208,"VINGTIEME, imposition",Economie politique,499,"Il n'y a pas-là d'arbitraire, ni d'acception, ..."
261630,17,3208,"VINGTIEME, imposition",Economie politique,500,"Et voilà toute l'affaire des finances, sans ve..."
261631,17,3208,"VINGTIEME, imposition",Economie politique,501,Il est aisé de sentir que ce cadastre pourroit...


In [24]:
df_sample_pred = predict_dataframe(df_sample, TEXT_COLUMN, batch_size=16)

Predicting:   0%|          | 0/15940 [00:00<?, ?batch/s]

/var/folders/qm/v_b1md29221_cnpcxf5qc43c0000gn/T/ipykernel_15010/317364521.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['predicted_label_id'] = all_preds
/var/folders/qm/v_b1md29221_cnpcxf5qc43c0000gn/T/ipykernel_15010/317364521.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['predDomain'] = df['predicted_label_id'].apply(lambda x: model.config.id2label[x])


In [25]:
df_sample.head()

,volume,numero,head,srcDomain,paragrapId,text,predicted_label_id,predDomain
0,1,1,Title Page,unclassified,1,"ENCYCLOPÉDIE,DICTIONNAIRE RAISONNÉDES SCIENCES...",12,Grammaire
1,1,1,Title Page,unclassified,2,"Mis en ordre & publié par M. DIDEROT, de l'Aca...",14,Histoire
3,1,1,Title Page,unclassified,4,"A PARIS,Chez BRIASSON, rue Saint Ja...",10,Commerce
7,1,2,A MONSEIGNEUR LE COMTE D'ARGENSON,unclassified,3,L'autorité suffit à un Ministre pour lui attir...,14,Histoire
12,1,3,DISCOURS PRÉLIMINAIRE DES EDITEURS,unclassified,2,"L'Encyclopédie que nous présentons au Public, ...",6,Belles-lettres


In [26]:
df.loc[df_sample_pred.index, 'predDomain'] = df_sample_pred['predDomain']

In [27]:
df

,volume,numero,head,srcDomain,paragrapId,text,predDomain
0,1,1,Title Page,unclassified,1,"ENCYCLOPÉDIE,DICTIONNAIRE RAISONNÉDES SCIENCES...",Grammaire
1,1,1,Title Page,unclassified,2,"Mis en ordre & publié par M. DIDEROT, de l'Aca...",Histoire
2,1,1,Title Page,unclassified,3,"Tantùm series juncturaque pollet,Tantùm de med...",NaN
3,1,1,Title Page,unclassified,4,"A PARIS,Chez BRIASSON, rue Saint Ja...",Commerce
4,1,1,Title Page,unclassified,5,M. DCC. L I.AVEC APPROBATION ET PRIVILEGE DU ROY.,NaN
...,...,...,...,...,...,...,...
261628,17,3208,"VINGTIEME, imposition",Economie politique,498,Elle seroit déposée dans chaque communauté com...,Droit Jurisprudence
261629,17,3208,"VINGTIEME, imposition",Economie politique,499,"Il n'y a pas-là d'arbitraire, ni d'acception, ...",Droit Jurisprudence
261630,17,3208,"VINGTIEME, imposition",Economie politique,500,"Et voilà toute l'affaire des finances, sans ve...",Politique
261631,17,3208,"VINGTIEME, imposition",Economie politique,501,Il est aisé de sentir que ce cadastre pourroit...,Droit Jurisprudence


In [28]:
filepath = '/Users/lmoncla/Nextcloud/GEODE/GEODE - Partage consortium/Corpus/EDdA/EDdA_dataset_paragraphs_pred40domains_260220.tsv'
df.to_csv(os.path.join(filepath), sep="\t", index=False)

In [60]:
path = os.path.join('..', 'data', '1743_LeRobert')
df.to_csv(os.path.join(path, "Trevoux1743_html_macro_pred_"+suffix+".tsv"), sep="\t", index=False)


In [ ]:
path = os.path.join('..', 'data', '1743_LeRobert')
df.to_csv(os.path.join(path, "Trevoux1743_paragraphs_macro_pred_"+suffix+".tsv"), sep="\t", index=False)
df.to_excel(os.path.join(path, "Trevoux1743_paragraphs_macro_pred_"+suffix+".xlsx"), index=False)